### Agora vamos pegar as cotações

In [47]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

# para rodar o chrome em 2º plano
from selenium.webdriver.chrome.options import Options
chrome_options = Options()
chrome_options.headless = True 
navegador = webdriver.Chrome(options=chrome_options)
# exclui até aqui para tirar o chrome em 2º plano

# Exibe o Chrome
# navegador = webdriver.Chrome()

# Cotação do dolar

# 1 - entrar no google
navegador.get("https://www.google.com/")

# 2 - digitar e pesquisar por "cotação dolar"
navegador.find_element(by = By.XPATH, value = '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação dolar", Keys.ENTER)

# 3 - obter o valor exibido na página e armazenar esse valor em uma variável de controle
cotacao_dolar = navegador.find_element(by = By.XPATH, value = '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")


# Cotação do euro

navegador.get("https://www.google.com/")
navegador.find_element(by = By.XPATH, value = '/html/body/div[1]/div[3]/form/div[1]/div[1]/div[1]/div/div[2]/input').send_keys("cotação euro", Keys.ENTER)

cotacao_euro = navegador.find_element(by = By.XPATH, value = '//*[@id="knowledge-currency__updatable-data-column"]/div[1]/div[2]/span[1]').get_attribute("data-value")

# Cotação do ouro

# 1 - entrar em https://www.melhorcambio.com/ouro-hoje
navegador.get("https://www.melhorcambio.com/ouro-hoje")

# 2 - obter o valor exibido na página e armazenar em uma var
cotacao_ouro = navegador.find_element(by = By.XPATH, value = '//*[@id="comercial"]').get_attribute("value")

# 3 - o valor da cotação é exibido com virgula (ex: 322,18), então devemos converter essa virgula em ponto.
cotacao_ouro = cotacao_ouro.replace(",", ".")

display(cotacao_dolar)
display(cotacao_euro)
display(cotacao_ouro)

navegador.close()

'5.1175999999999995'

'5.604098448'

'321.76'

### Agora vamos atualiza a nossa base de preços com as novas cotações

- Importando a base de dados

In [48]:
import pandas as pd

tabela = pd.read_excel("Produtos.xlsx")
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5,4999.95,1.40,6999.930
1,Carro Renault,4500.00,Euro,6,27000.00,2.00,54000.000
2,Notebook Dell,899.99,Dólar,5,4499.95,1.70,7649.915
3,IPhone,799.00,Dólar,5,3995.00,1.70,6791.500
4,Carro Fiat,3000.00,Euro,6,18000.00,1.90,34200.000
5,Celular Xiaomi,480.48,Dólar,5,2402.40,2.00,4804.800
6,Joia 20g,20.00,Ouro,350,7000.00,1.15,8050.000


- Atualizando os preços e o cálculo do Preço Final

In [49]:
# 1 - atualizando cotação

# Dolar
tabela.loc[tabela["Moeda"] == "Dólar", "Cotação"] = float(cotacao_dolar)

# Euro
tabela.loc[tabela["Moeda"] == "Euro", "Cotação"] = float(cotacao_euro)

# Ouro
tabela.loc[tabela["Moeda"] == "Ouro", "Cotação"] = float(cotacao_ouro)

# 2 - atualizar preço de compra (p. original * cotacao)
tabela["Preço de Compra"] = tabela["Preço Original"] * tabela["Cotação"]

# 3 - atualizar preço de venda (p. de compra * margem)
tabela["Preço de Venda"] = tabela["Preço de Compra"] * tabela["Margem"]
display(tabela)

,Produtos,Preço Original,Moeda,Cotação,Preço de Compra,Margem,Preço de Venda
0,Câmera Canon,999.99,Dólar,5.117600,5117.548824,1.40,7164.568354
1,Carro Renault,4500.00,Euro,5.604098,25218.443016,2.00,50436.886032
2,Notebook Dell,899.99,Dólar,5.117600,4605.788824,1.70,7829.841001
3,IPhone,799.00,Dólar,5.117600,4088.962400,1.70,6951.236080
4,Carro Fiat,3000.00,Euro,5.604098,16812.295344,1.90,31943.361154
5,Celular Xiaomi,480.48,Dólar,5.117600,2458.904448,2.00,4917.808896
6,Joia 20g,20.00,Ouro,321.760000,6435.200000,1.15,7400.480000


### Agora vamos exportar a nova base de preços atualizada

In [50]:
tabela.to_excel("Produtos 2022.xlsx", index = False)